In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


In [5]:
df=pd.read_csv("Threats (1).csv")
print(df.head())

   Unnamed: 0     id       dur  ... is_sm_ips_ports attack_cat label
0       40000  40001  0.012996  ...               0     Normal     0
1       40001  40002  0.004119  ...               0     Normal     0
2       40002  40003  0.000988  ...               0     Normal     0
3       40003  40004  3.415787  ...               0     Normal     0
4       40004  40005  0.193943  ...               0     Normal     0

[5 rows x 46 columns]


In [6]:
def frequency_encoding(df,categorial_columns):
    encoding={}
    for col in categorial_columns:
        freq=df[col].value_counts()
        df[col]=df[col].map(freq)
        encoding[col]=freq
    return df,encoding


In [12]:
freq = {}
df.drop(['Unnamed: 0','id'], axis = 'columns' ,inplace = True)
df,freq = frequency_encoding(df,['proto','service','state'])
labels = LabelEncoder()
df['attack_cat'] = labels.fit_transform(df['attack_cat'])

predict_feature = df['attack_cat'].to_numpy()
df.drop(['attack_cat'], axis = 'columns', inplace = True)
features =  df.to_numpy()

In [13]:
def KFold_crossval(x,y,clsr):
    kfold = KFold(n_splits = 6)
    accuracy = []
    for idx_train,idx_test in kfold.split(x,y):
        clsr.fit(x[idx_train],y[idx_train])
        y_predict = clsr.predict(x[idx_test])
        acc = accuracy_score(y[idx_test],y_predict)
        accuracy.append(acc)
    print(f"Average Accuracy = {sum(accuracy)/len(accuracy)}")
    print(accuracy)

In [14]:
# Random forest
rfc = RandomForestClassifier(n_estimators = 10)
KFold_crossval(features,predict_feature,rfc)

Average Accuracy = 0.8217991014357882
[0.8318925389014497, 0.7347608281243073, 0.7347608281243073, 0.7989980937181362, 0.8963957973134725, 0.9339865224330555]


In [15]:
# Bagging
bagging = BaggingClassifier(n_estimators = 10)
KFold_crossval(features,predict_feature,bagging)

Average Accuracy = 0.8227079249719749
[0.8331781708560535, 0.7371990956244181, 0.73289887839695, 0.798776433036308, 0.8982134149044643, 0.9359815570136549]


In [16]:
# Boosting - Histogram Gradient Boosting
hgbc = HistGradientBoostingClassifier()
KFold_crossval(features,predict_feature,hgbc)

Average Accuracy = 0.8301926666732338
[0.8400939841290952, 0.7507203972159419, 0.7477944762158089, 0.8078201888549009, 0.8988340648135834, 0.9358928888100727]


In [17]:
# Boosting - Ada Boost
abc = AdaBoostClassifier(n_estimators = 10)
KFold_crossval(features,predict_feature,abc)

C:\Users\mahek\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mahek\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mahek\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\mahek\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use t

Average Accuracy = 0.6914619460522738
[0.6716761980759853, 0.5877997960721727, 0.573391851753336, 0.678193022121736, 0.7950082014452277, 0.8427026068451853]


: 